## LSTM 셀 구현
지금까지 RNN을 살펴보았다면 이제 LSTM 셀과 LSTM 계층을 살펴보겠습니다. 이번 예제에서 사용할 데이터셋은 MNIST입니다.

MNIST는 손으로 쓴 숫자 이미지(0~9까지 값을 갖는 고정 크기 이미지 (28px x 28px))들로 구성되어 있습니다.

먼저 필요한 라이브러리들을 호출합니다.

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset

from torch.autograd import Variable
from torch.nn import Parameter # 파라미터 목록을 가지고 있는 라이브러리
from torch import Tensor
from torch.utils.data import DataLoader

import torch.nn.functional as F

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(125)

print(device)


cuda


데이터에 대한 전처리를 합니다. 평균과 표준편차에 맞게 데이터를 정규화하기 위한 코드입니다.

In [3]:
mnist_fransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5),(1.0)) # 평균을 0.5, 표준편차를 1.0으로 데이터 정규화
])

`torchvision.datasets` 에서 제공하는 데이터셋 중 MNIST 데이터셋을 내려받습니다.

In [4]:
train_dataset = dataset.MNIST(root='./MNIST_DATASET', transform=mnist_fransform, train=True, download=True)
validation_dataset = dataset.MNIST(
    root='./MNIST_DATASET', transform=mnist_fransform, train=False, download=True)

test_dataset = dataset.MNIST(
    root='./MNIST_DATASET', transform=mnist_fransform, train=False, download=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 19395944.46it/s]


Extracting ./MNIST_DATASET\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 9663796.87it/s]

Extracting ./MNIST_DATASET\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 6983886.52it/s]


Extracting ./MNIST_DATASET\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST_DATASET\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./MNIST_DATASET\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST_DATASET\MNIST\raw



* MNIST 데이터셋을 내려받기 위해 필요한 파라미터는 다음과 같습니다.

```py
train_dataset = dataset.MNIST(root='./MNIST_DATASET', transform=mnist_fransform, train=True, download=True)
```

* `root` : MNIST를 내려받을 위치 지정
* `transform` : 앞에서 정의했던 데이터 전처리 적용
* `train` : `True`로 설정할 경우 훈련용 데이터셋을 가져오지만, `False`로 설정할 경우 테스트용 데이터셋을 가져옵니다.
* `download` : `True`로 설정될 경우 내려받으려는 위치에 MNIST 파일이 없으면 내려받지만 파일이 있다면 내려받지 않습니다.

`dataloader`를 이용하여 내려받은 MNIST 파일을 메모리로 불러옵니다. 단, train_loader, valid_loader, test_loader 가 호출될 때 메모리로 불러온다는 점에 주의하세요

In [5]:
batch_size = 64

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
valid_loader = DataLoader(
    dataset=validation_dataset,
    batch_size=batch_size,
    shuffle=True
)
train_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)


배치 크기 및 에포크 등 변수에 대한 값을 지정합니다.

In [6]:
batch_size = 100
n_iters = 6000
num_epoch = n_iters / (len(train_dataset) / batch_size)
num_epoch = int(num_epoch)

print(num_epoch)
print(len(train_dataset))

10
60000


LSTM 셀에 대한 네트워크를 구축합니다. 모델의 전반적인 네트워크가 아닌 LSTM 셀에 집중한 네트워크입니다.

In [7]:
# class LSTMcell(nn.Module):
#     def __init__(self, input_size, hidden_size, bias=True):
#         super(LSTMcell, self).__init__()
#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.bias = bias
#         self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
#         self.h2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
#         self.reset_parameters()

#     def reset_parameters(self): # 모델의 파라미터 초기화
#         std = 1.0 / math.sqrt(self.hidden_size)
#         for w in self.parameters():
#             w.data.uniform_(-std, std)
    
#     def forward(self, x, hidden):
#         hx, cx = hidden
#         x = x.view(-1, x.size(1))

#         gates = self.x2h(x) + self.h2h(hx)
#         gates = gates.squeeze()
#         ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

#         ingate = F.sigmoid(ingate) # 입력게이트에 시그모이드 활성화 함수 적용
#         forgetgate = F.sigmoid(forgetgate) # 망각 게이트에 시그모이드 활성화 함수 적용
#         cellgate = F.tanh(cellgate) # 셀 게이트에 하이퍼볼릭 탄젠트 활성화 함수 적용
#         outgate = F.sigmoid(outgate) # 출력 게이트에 시그모이드 활성화 함수 적용

#         cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
#         hy = torch.mul(outgate, F.tanh(cy))
#         return(hy, cy)

In [28]:
class LSTMcell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMcell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias) 
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, hidden):
        hx, cx = hidden
        x = x.view(-1, x.size(1))

        gates = self.x2h(x) + self.h2h(hx)
        gates = gates.squeeze()
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)

        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        return (hy, cy)


```py
self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
```
```py
self.h2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
```

에서 `4 * hidden_size`가 사용되고 있는 이유에 대해 생각해 볼 필요가 있습니다. 왜 은닉층의 뉴런/유닛에 4를 곱할까요?

그 답을 알기 위해서는 다음 그림을 먼저 이해해야 합니다.

![](../Static/403_1.jpg)

그림과 같이 LSTM에서 중요한 것은 게이트입니다. 게이트는 `망각`, `입력`, `셀`, `출력` 으로 구성되며 이 모든 게이트는 다음과 같이 구할 수 있습니다.
```py
gates = F.linear(input, w_ih, b_ih) + F.linear(hx, w_hh, b_hh)
```

* `input` 입력층으로 입력되는 훈련 데이터셋의 특성(feature) 수(칼럼 개수)
* `w_ih` 입력층과 은닉층 사이의 가중치
* `b_ih` 입력층과 은닉층 사이의 바이어스
* `hx` 은닉층의 뉴런/유닛 개수(은닉층의 특성(feature) 수)
* `w_hh` 은닉층과 은닉층 사이의 가중치
* `b_hh` 은닉층 사이의 바이어스

이렇게 계산된 게이트(gates)는 `gates.chunk(4,1)`에 의해 네 개로 쪼개져서 각각 망각,입력,셀,출력 게이트를 의미하는 변수에 저장됩니다. 즉, gates가 네 개로 쪼개지는 상황이기 때문에 4가 곱해졌던 것입니다. 일반적으로 바이어스도 4를 곱해 주지만 예제에서는 은닉층의 뉴런/유닛 개수만 4를 곱해 주었습니다.

또한, 여기에서 사용되는 `torch.chunk()` 사용 방법은 다음과 같습니다.

`torch.chunk`는 텐서를 쪼갤 때 사용되는 함수이며 파라미터는 다음과 같습니다.

```py
ingate, forgetgate, cellgate, outgate = gates.chunk(4,1)
```
* 첫 번째 파라미터 : 텐서를 몇개로 쪼갤지 설정합니다.
* 두 번쨰 파라미터 : 어떤 차원을 기준으로 쪼갤지를 결정합니다. dim=1 이므로 열 단위로 텐서를 분할하겠다는 의미입니다.

```py
w.data.uniform_(-std, std)
```

`uniform()`은 난수를 위해 사용합니다. 난수 사용은 uniform() 외에도 randint(), random() 등이 있는데 사용 방법은 다음과 같습니다.

```py
from random import *

ri = randint(1, 10) # 1부터 10 사이의 임의의 정수
rd = random() # 0부터 1 사이의 임의의 실수(float)
ui = uniform(1, 10) # 1부터 10 사이의 임의의 실수(float)
rr = randrange(1, 10, 2) # 1부터 10사이를 2씩 건너뛴 임의의 정수 (ex 1,3,5,7,9)
```

`torch.squeeze()`는 텐서의 차원을 줄이고자 할 때 사용합니다. 사용방법은 다음과 같습니다.

```py
import torch
x = torch.FloatTensor([[1], [2]]) # (2 x 1) 크기의 2차원 텐서 생성
x.squeeze() # squeeze()가 적용되어 1차원으로 축소
```

* 하나의 LSTM 셀을 통과하면 셀($c_t$) 상태와 은닉 상태($h_t$)가 출력으로 주어집니다. 이때 셀 상태는 입력, 망각, 셀 게이트에 의해 계산되며, 은닉 상태는 출력 게이트에 의해 계산됩니다. 여기에서 사용되는 `torch.mul()`은 텐서에 곱셈을 할 때 사용합니다. 사용방법은 다음과 같습니다.

```py
import torch
x = torch.FloatTensor([[1], [2]]) # (2 x 1) 크기의 2차원 텐서 생성
torch.mul(x, 3) # x라는 텐서의 원소에 3을 곱합니다.
```

앞에서 게이트 위주의 셀에 대해 살펴보았다면 이번에는 전반적인 네트워크를 구성해 보겠습니다.

In [29]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim # 은닉층의 뉴런/유닛 개수
        self.layer_dim = layer_dim

        self.lstm = LSTMcell(input_dim, hidden_dim, layer_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device=device))
        # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 형태를 갖는 은닉 상태를 0으로 초기화
        c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device=device))
        # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 형태를 갖는 은닉 상태를 0으로 초기화

        outs = []
        cn = c0[0, :, :] # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 크기를 갖는 셀 상태에 대한 텐서
        hn = h0[0, :, :]  # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 크기를 갖는 셀 상태에 대한 텐서

        for seq in range(x.size(1)): # 셀 계층을 반복해서 쌓아 올림
            hn, cn = self.lstm(x[:,seq,:], (hn,cn)) # 은닉 상태(hn)와 셀 상태를 lstmcell에 적용한 결과를 다시 hn, cn에 저장
            outs.append(hn)

        out = outs[-1].squeeze()
        out = self.fc(out)
        return out


`LSTM 셀`은 앞에서 정의한 함수를 불러오는 부분으로 파라미터는 다음과 같습니다.
```py
self.lstm(input_dim, hidden_dim, layer_dim)
```

* input_dim 입력에 대한 특성 수 (칼럼 갯수)
* hidden_dim 은닉층의 뉴런 개수
* layer_dim 은닉층의 계층 개수

옵티마이저와 손실 함수를 지정합니다.

In [30]:
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 10

model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, layer_dim=layer_dim, output_dim=output_dim).to(device)

criterion = nn.CrossEntropyLoss()
learning_rate = 1e-1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



훈련 데이터셋을 이용하여 모델을 학습시키고, 검증 데이터셋을 이용하여 모델 성능을 확인해봅니다.

In [32]:
seq_dim = 28
loss_list = []
iter = 0

for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader): # 훈련 데이터셋을 이용한 모델 학습
        images = Variable(images.view(-1, seq_dim, input_dim).to(device))
        labels = Variable(labels.to(device))

    optimizer.zero_grad()
    outputs = model(images).to(device)
    loss = criterion(outputs, labels) # 손실 함수를 이용하여 오차 계산
    loss.backward()
    optimizer.step() # 파라미터업데이트
    loss_list.append(loss.item())
    iter += 1

    if iter % 500 == 0: # 정확도 계산
        correct = 0
        total = 0
        for images, labels in valid_loader: # 검증 데이터셋을 이용한 모델 성능 검증
            images = Variable(images.view(-1, seq_dim, input_dim).to(device))
        
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1) # 모델을 통과한 결과의 최댓값으로부터 예측 결과 가져오기

            total += labels.size(0) # 총 레이블 수
            correct += (predicted.cpu() == labels.cpu()).sum()

        accuracy = 100 * correct / total
        print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
